<a href="https://colab.research.google.com/github/smathews88/nlp/blob/main/05_NLP_LLM_FineTune_HuggingFace_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

def generate_sample_chat_data(num_conversations):
    data = []

In [ ]:
import json

def convert_chat_data_to_text(json_file, txt_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    with open(txt_file, 'w') as f:
        for chat in data:
            for message in chat['messages']:
                f.write(f"{message['role']}: {message['content']}\n")
            f.write("\n")  # Separate different chats with a newline

convert_chat_data_to_text('/content/drive/MyDrive/Colab Notebooks/dataset/ecommerce_chat_data.json', '/content/drive/MyDrive/Colab Notebooks/dataset/ecommerce_chat_data.txt')



In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Prepare dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=10
    )

# Load your dataset
train_dataset = load_dataset('/content/drive/MyDrive/Colab Notebooks/dataset/ecommerce_chat_data.txt', tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune model
trainer.train()

# Save model and tokenizer
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model")


Step,Training Loss


('/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model/added_tokens.json')

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Load fine-tuned model and tokenizer
model_name = "/content/drive/MyDrive/Colab Notebooks/model/fine_tuned_model"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

class Message(BaseModel):
    text: str

@app.post("/generate/")
async def generate_text(message: Message):
    inputs = tokenizer.encode(message.text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"response": text}



ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-19' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-169ff20402d0>", line 7, in <cell line: 7>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/u

In [ ]:
!ngrok authtoken '2iCXRwV7vMAi5f7ILDN3vGnColf_3inm59wgHUyHUmPcyGNrZ'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://485f-35-201-146-163.ngrok-free.app


INFO:     Started server process [23005]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate HTTP/1.1" 307 Temporary Redirect
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "OPTIONS /generate/ HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate/ HTTP/1.1" 200 OK
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate HTTP/1.1" 307 Temporary Redirect
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "OPTIONS /generate/ HTTP/1.1" 200 OK


...

INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate/ HTTP/1.1" 200 OK
INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate HTTP/1.1" 307 Temporary Redirect


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     2001:8f8:1921:14c3:40ab:8ccf:f5fa:2363:0 - "POST /generate/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23005]


KeyboardInterrupt: 